In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'hole_diameter'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['HoleDiameter']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

332


In [5]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [7]:
pat_frac=rf'''(?i).{0,10}\d*.?\d+\/\d+.{0,3}diame?t?e?r?.{0,10}|()'''         
df['name'] = df['product_name'].apply(lambda x: re_extract(pat_frac, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pat_frac, str(x)))


pat=r'(?i)(.{0,10}\d*.?\d+\/\d+.)|()'
df['matches']=df['HoleDiameter'].apply(lambda x: re_extract(pat, str(x)))
custom=df[df['matches'].astype(str)!='[]']
custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|"','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'"\]',' in"]',str(x)))
matchcustom=custom[['external_id',curation_col]]
matchcustom

external_id Q:hole_diameter
323       11702      ["1/2 in"]
325       11706    ["1-1/2 in"]
326       11705    ["1-1/4 in"]
327       11703      ["3/4 in"]

In [8]:
two=df[df['matches'].astype(str)=='[]']
pat_diam=rf'''(?i)(.?.?.?.?.?.?\d.?.?.?.?.?hole.?Diam?e?t?e?r?(?! mounting)(?! abrasive ))|(hole.?Diam?e?t?e?r?.?.?\d.?.?.?.?.?.?.?)|()''' 
two['matches'] = two['long_desc'].apply(lambda x: re_extract(pat_diam, str(x)))
two[two['matches'].astype(str)!='[]']

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, HoleDiameter, name, ld, matches]
Index: []

# N/A

In [9]:
na=two[two['matches'].astype(str)=='[]']
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
print(len(na))

328


In [10]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))

matchcustom
matchna


In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [11]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [12]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
# looks_good('Kimball MidWest', attribute, df, matchex) 